<a href="https://colab.research.google.com/github/AsimAbbas12/album/blob/master/Train%20a%20CNN%20on%20the%20SVHN%20Dataset%20for%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import requests




In [ ]:
def download_svhn(filename, url):
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        response = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"{filename} downloaded.")
# Helper function to download the dataset if not already present


In [ ]:
train_url = "http://ufldl.stanford.edu/housenumbers/train_32x32.mat"
test_url = "http://ufldl.stanford.edu/housenumbers/test_32x32.mat"

train_file = "train_32x32.mat"
test_file = "test_32x32.mat"

download_svhn(train_file, train_url)
download_svhn(test_file, test_url)

# Download the SVHN dataset if not available

train_32x32.mat downloaded.
test_32x32.mat downloaded.


In [ ]:
svhn_train = loadmat(train_file)
svhn_test = loadmat(test_file)
# 1. Load the SVHN dataset


In [ ]:
# Extract images and labels
X_train = svhn_train['X']
y_train = svhn_train['y']
X_test = svhn_test['X']
y_test = svhn_test['y']



In [ ]:
print("Train images shape:", X_train.shape)  # (32, 32, 3, num_samples)
print("Train labels shape:", y_train.shape)
print("Test images shape:", X_test.shape)
print("Test labels shape:", y_test.shape)

# 2. Check the data shape

Train images shape: (32, 32, 3, 73257)
Train labels shape: (73257, 1)
Test images shape: (32, 32, 3, 26032)
Test labels shape: (26032, 1)


In [ ]:
# Reshape the data to match TensorFlow's expected input
X_train = np.moveaxis(X_train, -1, 0)
X_test = np.moveaxis(X_test, -1, 0)
print("Reshaped train images:", X_train.shape)  # Now (num_samples, 32, 32, 3)



Reshaped train images: (73257, 32, 32, 3)


In [ ]:
# 3. Normalize the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0



In [ ]:
# Correct labels: In the dataset, the label '10' represents '0', fix it
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0



In [ ]:
# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



In [ ]:
# 4. Split the train set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# 5. Define the CNN model


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Compile the model

In [ ]:

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

# 6. Train the model

Epoch 1/10
916/916 ━━━━━━━━━━━━━━━━━━━━ 134s 144ms/step - accuracy: 0.3747 - loss: 1.7973 - val_accuracy: 0.8042 - val_loss: 0.6475
Epoch 2/10
916/916 ━━━━━━━━━━━━━━━━━━━━ 140s 141ms/step - accuracy: 0.8001 - loss: 0.6782 - val_accuracy: 0.8630 - val_loss: 0.4625
Epoch 3/10
916/916 ━━━━━━━━━━━━━━━━━━━━ 119s 116ms/step - accuracy: 0.8420 - loss: 0.5316 - val_accuracy: 0.8774 - val_loss: 0.4094
Epoch 4/10
916/916 ━━━━━━━━━━━━━━━━━━━━ 126s 99ms/step - accuracy: 0.8609 - loss: 0.4703 - val_accuracy: 0.8777 - val_loss: 0.4074
Epoch 5/10
916/916 ━━━━━━━━━━━━━━━━━━━━ 142s 99ms/step - accuracy: 0.8743 - loss: 0.4312 - val_accuracy: 0.8914 - val_loss: 0.3671
Epoch 6/10
916/916 ━━━━━━━━━━━━━━━━━━━━ 144s 101ms/step - accuracy: 0.8814 - loss: 0.4020 - val_accuracy: 0.8957 - val_loss: 0.3555
Epoch 7/10
916/916 ━━━━━━━━━━━━━━━━━━━━ 143s 102ms/step - accuracy: 0.8904 - loss: 0.3715 - val_accuracy: 0.8931 - val_loss: 0.3638
Epoch 8/10
916/916 ━━━━━━━━━━━━━━━━━━━━ 145s 105ms/step - accuracy: 0.8948 - l

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

# 7. Evaluate the model

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# 8. Generate predictions

In [ ]:
cm = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.show()

# 9. Confusion Matrix

In [1]:
# 10. Visualize test samples with predicted and true labels
# Choose indices for samples to display
indices = [0, 1, 2, 3, 4]  # You can choose any indices of interest

# Plot the selected test samples
plt.figure(figsize=(15, 5))
for i, idx in enumerate(indices):
    plt.subplot(1, len(indices), i + 1)
    plt.imshow(X_test[idx])
    plt.title(f"True: {y_true[idx]}\nPred: {y_pred_classes[idx]}")
    plt.axis('off')

plt.show()

NameError: name 'plt' is not defined